In [26]:
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.axes as axs

sns.set()

# %load_ext nb_black
%load_ext lab_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


<IPython.core.display.Javascript object>

### gen purpose functions

In [27]:
def mis_set_num(set_):
    """obtain missing numeric values in set

    Args:
        set_ (set or list like object): set of numbers

    Returns:
        set: returns set of missing values in passed container
    """
    return sorted(set(range(set_[0], set_[-1])) - set(set_))


def convert_test_int(set_):
    """convert elemetnt to list

    Args:
        set_ (set or list like object): set of numbers

    Returns:
        set: returns warning if integer conversion fails
    """
    try:
        return [int(x) for x in set_]
    except:
        print("nan at position")


def date_to_datetime_input(df_):
    """generate SQL compliant date string input

    Args:
        df_ (DataFrame): input DataFrame
        datecol (series): datetime compliant pandas series

    Returns:
        series: return SQL compiant
    """
    return df_["date"].apply(lambda x: df_.datecol.str.replace("/", "-").str.split()[:2])
        

<IPython.core.display.Javascript object>

## Import the data using the relvant specifications

In [28]:
# REad the data
df = pd.read_csv(
    "{}/Data/crimes.csv".format("/".join(os.getcwd().split("/")[:-1])),
    delimiter="\t",
    dtype={
        "CrimeID": str,
        "beat": str,
        # "iucr": str,
        "district": str,
        "casenumber": str,
        # "date": str,
        # "block": str,
        "location": str,
    },
)
None

/tmp/ipykernel_57234/3994664303.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


<IPython.core.display.Javascript object>

### Missing values processing 

In [29]:
df.head()

,CrimeID,casenumber,date,block,iucr,primarytype,description,locationdescription,arrest,beat,district,latitude,longitude,location
0,1,JA118920,01/01/2017 01:00:00 AM,023XX N WAYNE AVE,1156,DECEPTIVE PRACTICE,ATTEMPT - FINANCIAL IDENTITY THEFT,RESIDENCE,false,1811,18,41.924198,-87.662071,"(41.924196311, -87.662069166)"
1,2,JA100052,01/01/2017 01:00:00 AM,090XX S BLACKSTONE AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,true,413,4,41.730709,-87.587997,"(41.730707788, -87.587998843)"
2,3,JA100367,01/01/2017 01:00:00 AM,049XX N WHIPPLE ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,1713,17,41.970776,-87.704872,"(41.97077465, -87.704873143)"
3,4,JA100092,01/01/2017 01:00:00 AM,023XX S CALIFORNIA AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,STREET,false,1034,10,41.848907,-87.695366,"(41.848905706, -87.695364781)"
4,5,JA494707,01/01/2017 01:00:00 AM,032XX N MILWAUKEE AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,APARTMENT,false,1732,17,41.940193,-87.725647,"(41.940192486, -87.725649965)"


<IPython.core.display.Javascript object>

### Initial misisng values check

In [30]:
# simple missing values check
df.isna().sum()

CrimeID                    0
casenumber                 3
date                       2
block                      0
iucr                       3
primarytype                3
description                3
locationdescription     3434
arrest                     3
beat                       3
district                   3
latitude               11583
longitude              11581
location               11581
dtype: int64

<IPython.core.display.Javascript object>

remove all instances that do not have a casenumber - resulting in 3 instances removed from the database


In [31]:
# those with missing casenumbers are unworkable
isna_index = df[df.casenumber.isna()].index
df.drop(df[df.casenumber.isna()].index, axis=0, inplace=True)

<IPython.core.display.Javascript object>

In [32]:
df.isna().sum()

CrimeID                    0
casenumber                 0
date                       0
block                      0
iucr                       0
primarytype                0
description                0
locationdescription     3431
arrest                     0
beat                       0
district                   0
latitude               11580
longitude              11580
location               11580
dtype: int64

<IPython.core.display.Javascript object>

### search for hidden NA or uncommen NaN values not detected by python
Simple Trick: Search for dataobjects by column which do not correspond with the correct data object in ths first place 

In [33]:
# find search set space for missing values
print(mis_set_num(df.CrimeID.astype(int).values))

missing_values = mis_set_num(df.CrimeID.astype(int).values)

[30000, 98000, 444444]


<IPython.core.display.Javascript object>

In [12]:
isna_index

Int64Index([30038, 98038, 444482], dtype='int64')

<IPython.core.display.Javascript object>

In [36]:
# find non vonvertables in CrimeId
convert_test_int(df.CrimeID.values)
None

<IPython.core.display.Javascript object>

the dropped indexes correspond with the removed observarions for missing case numbers

In [13]:
df[df.index.isin(missing_values)]

,CrimeID,casenumber,date,block,iucr,primarytype,description,locationdescription,arrest,beat,district,latitude,longitude,location
30000,29962,JB124174,01/16/2018 09:00:00 AM,007XX N SPAULDING AVE,0820,THEFT,$500 AND UNDER,RESIDENCE,false,1121,11,41.894493,-87.708969,"(41.894493874, -87.708967553)"
98000,97962,JA162833,02/22/2017 03:52:00 AM,027XX W 42ND ST,1020,ARSON,BY FIRE,VEHICLE NON-COMMERCIAL,false,921,9,41.817398,-87.692429,"(41.817398864, -87.692427281)"
444444,444406,JB389583,08/11/2018 11:20:00 PM,011XX N LARAMIE AVE,1365,CRIMINAL TRESPASS,TO RESIDENCE,RESIDENCE,false,1524,15,41.900894,-87.755745,"(41.900894005, -87.755743288)"


<IPython.core.display.Javascript object>

In [ ]:
df.CrimeID = df.CrimeID.astype("str", copy=True, errors="raise")

<IPython.core.display.Javascript object>

### Duplicates

In [17]:
df.drop_duplicates(keep="first", inplace=True, ignore_index=True)

<IPython.core.display.Javascript object>

dropping duplicates repoves 149 instances

In [9]:
# doublechecking confirms
df.drop_duplicates(["casenumber"]).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730748 entries, 0 to 730747
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CrimeID              730748 non-null  object 
 1   casenumber           730748 non-null  object 
 2   date                 730748 non-null  object 
 3   block                730748 non-null  object 
 4   iucr                 730748 non-null  object 
 5   primarytype          730748 non-null  object 
 6   description          730748 non-null  object 
 7   locationdescription  727321 non-null  object 
 8   arrest               730748 non-null  object 
 9   beat                 730748 non-null  object 
 10  district             730748 non-null  object 
 11  latitude             719169 non-null  float64
 12  longitude            719169 non-null  float64
 13  location             719169 non-null  object 
dtypes: float64(2), object(12)
memory usage: 83.6+ MB


<IPython.core.display.Javascript object>

In [21]:
df.drop_duplicates(["CrimeID"]).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730748 entries, 0 to 730747
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CrimeID              730748 non-null  object 
 1   casenumber           730748 non-null  object 
 2   date                 730748 non-null  object 
 3   block                730748 non-null  object 
 4   iucr                 730748 non-null  object 
 5   primarytype          730748 non-null  object 
 6   description          730748 non-null  object 
 7   locationdescription  727321 non-null  object 
 8   arrest               730748 non-null  object 
 9   beat                 730748 non-null  object 
 10  district             730748 non-null  object 
 11  latitude             719169 non-null  float64
 12  longitude            719169 non-null  float64
 13  location             719169 non-null  object 
dtypes: float64(2), object(12)
memory usage: 83.6+ MB


<IPython.core.display.Javascript object>

Conclusion: no duplicates in CrimeId and Casenumber

### Beat & District analysis

See whether beat and district have the right amount of entities (beat should have length 4; district length 2)
Adjust for "0" preceeding



In [66]:
# start with districts
# df[df.district.str.len() < 2]

<IPython.core.display.Javascript object>

In [65]:
# Continue with districts
# df[df.beat.str.len() < 4]

<IPython.core.display.Javascript object>

In [50]:
# alternatively:
# 1. insert 0s in the front of each beat and district where length is smaller 4 or smaller 2 respectively
df["beat_"] = df["beat"].apply(lambda x: "{0:0>4}".format(x))
df["district_"] = df["district"].apply(lambda x: "{0:0>2}".format(x))

<IPython.core.display.Javascript object>

In [54]:
# 2) now check where the first two digits of beat_ does NOT comply with the two digits of district_
df[df.beat_.str[:2] != df.district_]

,CrimeID,casenumber,date,block,iucr,primarytype,description,locationdescription,arrest,beat,district,latitude,longitude,location,beat_,district_
5106,5068,JA308088,01/03/2017 12:01:00 AM,021XX W GIDDINGS ST,1582,OFFENSE INVOLVING CHILDREN,CHILD PORNOGRAPHY,RESIDENCE,false,1911,1,41.885700,-87.642029,"(41.885702079, -87.642031644)",1911,01
6303,6265,JD102780,01/03/2020 11:10:00 AM,097XX W KENNEDY EXPY OB,0486,BATTERY,DOMESTIC BATTERY SIMPLE,HIGHWAY/EXPRESSWAY,true,1654,31,41.982494,-87.875580,"(41.98249319, -87.875577905)",1654,31
33085,33047,JE116040,01/17/2021 12:00:00 AM,005XX W OHARE ST,0530,ASSAULT,AGGRAVATED - OTHER DANGEROUS WEAPON,PARKING LOT / GARAGE (NON RESIDENTIAL),false,1653,31,41.965057,-87.879951,"(41.965057367, -87.879953326)",1653,31
59633,59595,JC288579,02/01/2017 12:00:00 AM,038XX S COTTAGE GROVE AVE,1754,OFFENSE INVOLVING CHILDREN,AGGRAVATED SEXUAL ASSAULT OF CHILD BY FAMILY M...,APARTMENT,false,212,3,41.783707,-87.594719,"(41.783707907, -87.594718494)",0212,03
61044,61006,JD429703,02/01/2020 12:00:00 AM,079XX S CHAMPLAIN AVE,1752,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,APARTMENT,false,624,11,41.870483,-87.719666,"(41.870481746, -87.719665179)",0624,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702838,702690,JB558892,12/16/2018 07:00:00 PM,026XX S DRAKE AVE,1754,OFFENSE INVOLVING CHILDREN,AGGRAVATED SEXUAL ASSAULT OF CHILD BY FAMILY M...,RESIDENCE,False,1032,31,41.968014,-87.818794,"(41.968013038, -87.818796103)",1032,31
707392,707244,JD464205,12/18/2020 05:00:00 AM,014XX W 72ND ST,0266,CRIMINAL SEXUAL ASSAULT,PREDATORY,RESIDENCE,True,734,6,41.754051,-87.659424,"(41.754052908, -87.659425103)",0734,06
721320,721172,JC152102,12/26/2018 12:13:00 PM,015XX E 87TH ST,1110,DECEPTIVE PRACTICE,BOGUS CHECK,SMALL RETAIL STORE,False,412,22,41.735855,-87.658737,"(41.73585412, -87.658739723)",0412,22
724433,724285,JB570460,12/28/2018 04:56:00 PM,049XX S KEDZIE AVE,0860,THEFT,RETAIL THEFT,SMALL RETAIL STORE,False,821,9,41.804962,-87.703888,"(41.804962024, -87.703884743)",0821,09


<IPython.core.display.Javascript object>

This returns another 79 porblematic instances

In [55]:
# those columns that contain problems will from now on be marked
df["marked"] = False
df.loc[df.index.isin(df[df.beat_.str[:2] != df.district_].index), "marked"] = True

<IPython.core.display.Javascript object>

In [73]:
# replace beat with beat_
df.beat = df.beat_
df.district = df.district_

df.drop(["beat_", "district_"], axis=1, inplace=True)

AttributeError: 'DataFrame' object has no attribute 'beat_'

<IPython.core.display.Javascript object>

### length of each instance 

Case number has a mandatory length of 8; 

See whether beat and district have the right amount of entities (beat should have length 4; district length 2)
Adjust for "0" preceeding

string is na: '', ' ',"N/A"

In [67]:
df.head()

,CrimeID,casenumber,date,block,iucr,primarytype,description,locationdescription,arrest,beat,district,latitude,longitude,location,beat_,district_,marked
0,1,JA118920,01/01/2017 01:00:00 AM,023XX N WAYNE AVE,1156,DECEPTIVE PRACTICE,ATTEMPT - FINANCIAL IDENTITY THEFT,RESIDENCE,false,1811,18,41.924198,-87.662071,"(41.924196311, -87.662069166)",1811,18,False
1,2,JA100052,01/01/2017 01:00:00 AM,090XX S BLACKSTONE AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,true,413,4,41.730709,-87.587997,"(41.730707788, -87.587998843)",0413,04,False
2,3,JA100367,01/01/2017 01:00:00 AM,049XX N WHIPPLE ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,1713,17,41.970776,-87.704872,"(41.97077465, -87.704873143)",1713,17,False
3,4,JA100092,01/01/2017 01:00:00 AM,023XX S CALIFORNIA AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,STREET,false,1034,10,41.848907,-87.695366,"(41.848905706, -87.695364781)",1034,10,False
4,5,JA494707,01/01/2017 01:00:00 AM,032XX N MILWAUKEE AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,APARTMENT,false,1732,17,41.940193,-87.725647,"(41.940192486, -87.725649965)",1732,17,False


<IPython.core.display.Javascript object>

### Check date 

DATETIME - format: YYYY-MM-DD HH:MI:SS.
TIMESTAMP - format: YYYY-MM-DD HH:MI:SS.

Case number has a mandatory length of 8; 

See whether beat and district have the right amount of entities (beat should have length 4; district length 2)
Adjust for "0" preceeding

string is na: '', ' ',"N/A"

In [74]:
df.date

0         01/01/2017 01:00:00 AM
1         01/01/2017 01:00:00 AM
2         01/01/2017 01:00:00 AM
3         01/01/2017 01:00:00 AM
4         01/01/2017 01:00:00 AM
                   ...          
730895    12/31/2021 12:38:00 AM
730896    12/31/2021 12:41:00 AM
730897    12/31/2021 12:41:00 PM
730898    12/31/2021 12:49:00 AM
730899    12/31/2021 12:55:00 AM
Name: date, Length: 730897, dtype: object

<IPython.core.display.Javascript object>

In [76]:
# simple conversion
# now convert date
df["date_time"] = pd.to_datetime(df.date.astype(str), errors="coerce")


,CrimeID,casenumber,date,block,iucr,primarytype,description,locationdescription,arrest,beat,district,latitude,longitude,location,marked,date_time
34,23,JB235763,,016XX W PRATT BLVD,1563,SEX OFFENSE,CRIMINAL SEXUAL ABUSE,APARTMENT,,2432,24,NaN,NaN,NaN,False,NaT
35,23,JB235763,,016XX W PRATT BLVD,1563,SEX OFFENSE,CRIMINAL SEXUAL ABUSE,APARTMENT,,2432,24,NaN,NaN,NaN,False,NaT
161,123,JA100380,,078XX S MORGAN ST,0810,THEFT,OVER $500,RESIDENCE,,0612,06,41.751896,-87.648926,"(41.751896943, -87.648926093)",False,NaT
360,322,JA469768,,076XX S VERNON AVE,1122,DECEPTIVE PRACTICE,COUNTERFEIT CHECK,RESIDENCE,,0624,06,41.755615,-87.613823,"(41.755615044, -87.613820045)",False,NaT
1161,1123,JB346519,,080XX S KEDVALE AVE,0553,ASSAULT,AGGRAVATED PO: OTHER DANG WEAP,STREET,,0834,08,41.747280,-87.725410,"(41.747280312, -87.725413197)",False,NaT
1360,1322,JD348240,,078XX S WINCHESTER AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,,0611,06,41.751614,-87.671959,"(41.751611954, -87.671958451)",False,NaT
1544,1506,JC100965,,035XX W FLOURNOY ST,1365,CRIMINAL TRESPASS,TO RESIDENCE,RESIDENCE,,1133,11,41.872681,-87.714066,"(41.872680029, -87.714066886)",False,NaT
11360,11322,JC107233,,012XX N HERMITAGE AVE,0810,THEFT,OVER $500,PARKING LOT/GARAGE(NON.RESID.),,1424,14,41.905903,-87.670692,"(41.905903969, -87.670691663)",False,NaT
11544,11506,JC106740,,025XX N DRAKE AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE-GARAGE,,1413,14,41.927116,-87.715836,"(41.92711605, -87.715837448)",False,NaT
15023,14985,JC109833,,077XX S YATES BLVD,0497,BATTERY,AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON,APARTMENT,,0421,04,41.754723,-87.566292,"(41.754723038, -87.566292236)",False,NaT


<IPython.core.display.Javascript object>

In [96]:
# create function that changes datetime inputs into the correct format
def date_to_datetime_input(df_):
    """generate SQL compliant date string input

    Args:
        df_ (DataFrame): input DataFrame
        datecol (series): datetime compliant pandas series

    Returns:
        series: return SQL compiant
    """
    return df_["date"].apply(lambda x: df.date.str.replace("/", "-").str.split()[:2])

<IPython.core.display.Javascript object>

In [98]:
# date_to_datetime_input(df)

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [79]:
df[df["date_time"].isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 34 to 624180
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CrimeID              16 non-null     object        
 1   casenumber           16 non-null     object        
 2   date                 16 non-null     object        
 3   block                16 non-null     object        
 4   iucr                 16 non-null     object        
 5   primarytype          16 non-null     object        
 6   description          16 non-null     object        
 7   locationdescription  15 non-null     object        
 8   arrest               16 non-null     object        
 9   beat                 16 non-null     object        
 10  district             16 non-null     object        
 11  latitude             14 non-null     float64       
 12  longitude            14 non-null     float64       
 13  location             14 non-null

<IPython.core.display.Javascript object>

an additional 16 entries do not contain dates; this is pivotal so they have to be removed

In [100]:
df.date_time_str = df.date_time.astype(str)

/tmp/ipykernel_57234/2542554214.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.date_time_str = df.date_time.astype(str)


<IPython.core.display.Javascript object>

### Outliers in locatioN!

In [ ]:
df[df.isnull() == True]

In [102]:
df[df.isnull()]

,CrimeID,casenumber,date,block,iucr,primarytype,description,locationdescription,arrest,beat,district,latitude,longitude,location,marked,date_time
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
730896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
730897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
730898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


<IPython.core.display.Javascript object>

In [106]:
df[~pd.notnull(df)]

,CrimeID,casenumber,date,block,iucr,primarytype,description,locationdescription,arrest,beat,district,latitude,longitude,location,marked,date_time
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
730896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
730897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
730898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


<IPython.core.display.Javascript object>

In [107]:
df.dropna(how="all")

,CrimeID,casenumber,date,block,iucr,primarytype,description,locationdescription,arrest,beat,district,latitude,longitude,location,marked,date_time
0,1,JA118920,01/01/2017 01:00:00 AM,023XX N WAYNE AVE,1156,DECEPTIVE PRACTICE,ATTEMPT - FINANCIAL IDENTITY THEFT,RESIDENCE,false,1811,18,41.924198,-87.662071,"(41.924196311, -87.662069166)",False,2017-01-01 01:00:00
1,2,JA100052,01/01/2017 01:00:00 AM,090XX S BLACKSTONE AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,true,0413,04,41.730709,-87.587997,"(41.730707788, -87.587998843)",False,2017-01-01 01:00:00
2,3,JA100367,01/01/2017 01:00:00 AM,049XX N WHIPPLE ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,1713,17,41.970776,-87.704872,"(41.97077465, -87.704873143)",False,2017-01-01 01:00:00
3,4,JA100092,01/01/2017 01:00:00 AM,023XX S CALIFORNIA AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,STREET,false,1034,10,41.848907,-87.695366,"(41.848905706, -87.695364781)",False,2017-01-01 01:00:00
4,5,JA494707,01/01/2017 01:00:00 AM,032XX N MILWAUKEE AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,APARTMENT,false,1732,17,41.940193,-87.725647,"(41.940192486, -87.725649965)",False,2017-01-01 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730895,730747,JE493505,12/31/2021 12:38:00 AM,005XX E 32ND ST,031A,ROBBERY,ARMED - HANDGUN,RESIDENCE - YARD (FRONT / BACK),False,0211,02,41.836071,-87.612816,"(41.83607107, -87.612813156)",False,2021-12-31 00:38:00
730896,730748,JE493511,12/31/2021 12:41:00 AM,038XX W 68TH PL,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,0833,08,41.768700,-87.718941,"(41.76870088, -87.718942599)",False,2021-12-31 00:41:00
730897,730749,JF100517,12/31/2021 12:41:00 PM,038XX W 83RD PL,0820,THEFT,$500 AND UNDER,RESIDENCE - PORCH / HALLWAY,False,0834,08,41.741333,-87.717537,"(41.741332605, -87.717536356)",False,2021-12-31 12:41:00
730898,730750,JF100660,12/31/2021 12:49:00 AM,075XX S SEELEY AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,RESIDENCE,False,0611,06,41.756420,-87.674522,"(41.756418241, -87.674522275)",False,2021-12-31 00:49:00


<IPython.core.display.Javascript object>

### Reset the index